<a href="https://colab.research.google.com/github/sbooeshaghi/azucar/blob/main/analysis/293T/obs2/ec_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title import
import os

import matplotlib.pyplot as plt
from sklearn.metrics import rand_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
import json
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from collections import defaultdict
from scipy.io import mmread, mmwrite
from scipy.sparse import csr_matrix
from sklearn.neighbors import KDTree
from scipy.stats import entropy
from itertools import combinations
import sys
import gzip
from scipy.stats import entropy
from sklearn.mixture import GaussianMixture


def nd(arr):
    return np.asarray(arr).reshape(-1)

def yex(ax):
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

fsize=20

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [4]:
#@title mx index
def write_dict(fname, d):
    inv_d = {v: k for k, v in d.items()}
    with open(fname, 'w') as f:
        for idx in range(len(d)):
            f.write(f'{inv_d[idx]}\n')


def write_markers(fname, markers):
    with open(fname, 'w') as f:
        for k, v in markers.items():
            f.write(f'{k}\t')
            n = len(v)
            for idx, i in enumerate(v):
                f.write(f'{i}')
                if idx < n - 1:
                    f.write(',')
            f.write('\n')


def read_markers(fname,
                 markers_ec=defaultdict(list),
                 celltype=defaultdict(),
                 marker_genes=defaultdict()):
    with open(fname, 'r') as f:
        for idx, line in enumerate(f.readlines()):
            ct, genes = line.strip().split('\t')
            celltype[ct] = idx

            # two things
            # 1. make marker_genes list
            # 2. make markers_ec
            for g in genes.split(','):
                gidx = len(marker_genes)

                # check if the gene has been added already
                if g in marker_genes.keys():  # gene repeated
                    gidx = marker_genes[g]
                else:
                    marker_genes[g] = gidx

                # for the cell type index, add the marker gene index
                markers_ec[celltype[ct]].append(marker_genes[g])

            # sort the marker genes
            markers_ec[celltype[ct]] = sorted(markers_ec[celltype[ct]])


def ec_index(markers_fname, 
             out_markers_ec_fn, 
             out_groups_fn, 
             out_marker_genes_fn):
    markers_ec = defaultdict(list)
    celltypes = defaultdict()
    marker_genes = defaultdict()
    read_markers(markers_fname, markers_ec, celltypes, marker_genes)

    write_markers(out_markers_ec_fn, markers_ec)
    write_dict(out_groups_fn, celltypes)
    write_dict(out_marker_genes_fn, marker_genes)

In [3]:
test_markers = {
    "g1": ["tag1", "tag3"],
    "g2": ["tag3", "tag4"],
    "g3" : ["tag2", "tag5"]
}

write_markers("markers.txt", test_markers)

In [7]:
#@title test mx index
# index the markers -> markers.ec marker_genes.txt groups.txt
ec_index("./markers.txt", 
         "./markers.ec", 
         "./groups.txt", 
         "./marker_genes.txt")

!cat markers.txt
!cat markers.ec
!cat marker_genes.txt

g1	tag1,tag3
g2	tag3,tag4
g3	tag2,tag5
0	0,1
1	1,2
2	3,4
tag1
tag3
tag4
tag2
tag5
